# Open Weather Map
Exploration of openweathermap.org API.
## Prerequisits

In [1]:
import toml
import requests
import pandas as pd

def del_lst(x):
    return x[0]

conf = toml.load('../../conf.toml')
owm = conf['owm']

url = 'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lng}' \
      '&units=metric&APPID={key}'
url = url.format(lat=owm['lat'], lng=owm['lng'], key=owm['key'])

## API
Get data from API and safe json file.

In [2]:
response = requests.get(url)
data = response.json()
data.keys()

dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'current', 'minutely', 'hourly', 'daily', 'alerts'])

## Current Weather
- Timezone is UTC
- temp, feels_like, dew_point: °C
- pressure: hPa
- humidity, clouds: %
- uvi: UV Index
- visibility: m
- wind_speed: m / s
- wind_deg: °

In [3]:
keys = ['timezone', 'timezone_offset', 'current']
time = ['dt', 'sunrise', 'sunset']

cur = pd.DataFrame(list(map(data.get, keys)), keys).T
cur = cur.join(pd.DataFrame(cur.current[0], [0]))
cur = cur.join(pd.DataFrame(cur.weather[0], [0]))
cur = cur.drop(columns=['current', 'weather'])

cur['timezone_offset'] = pd.to_timedelta(cur.timezone_offset, 's')
cur[time] = cur[time].apply(pd.to_datetime, unit='s')
cur = cur.set_index('dt')
cur

,timezone,timezone_offset,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,rain,id,main,description,icon
dt,,,,,,,,,,,,,,,,,,,
2021-05-05 12:14:58,Europe/Berlin,0 days 02:00:00,2021-05-05 03:46:14,2021-05-05 19:05:35,8.21,3.35,996,81,5.15,0.89,75,8000,12.35,290,NaN,500,Rain,light rain,10d


### Alerts

In [4]:
# ale = pd.DataFrame(data['alerts'], ['german', 'english'])
#
# time = ['start', 'end']
#
# ale[time] = ale[time].apply(pd.to_datetime, unit='s')
# ale

## Minutely Data
- 1 hour
- 60 observations
- precipitation: mm

In [5]:
mnt = pd.DataFrame(data['minutely'])
mnt = mnt.set_index('dt')
mnt.index = pd.to_datetime(mnt.index, unit='s')
mnt

,precipitation
dt,
2021-05-05 12:15:00,0.1780
2021-05-05 12:16:00,0.1496
2021-05-05 12:17:00,0.1212
2021-05-05 12:18:00,0.0000
2021-05-05 12:19:00,0.0000
...,...
2021-05-05 13:11:00,0.0000
2021-05-05 13:12:00,0.0000
2021-05-05 13:13:00,0.0000


## Hourly Data
- 2 days
- 48 observations
- wind_gust: metre/sec
- pop: %
- rain: mm

In [6]:
hr = pd.DataFrame(data['hourly'])

hr = hr.set_index('dt')
hr.index = pd.to_datetime(hr.index, unit='s')

s = hr.rain.notna()
hr['precipitation'] = 0
hr.loc[s, 'precipitation'] = hr.loc[s, 'rain'].apply(lambda x: x.get('1h'))
hr = hr.drop(columns='rain')

hr = hr.join(pd.DataFrame(hr.weather.apply(del_lst).tolist(), hr.index))
hr = hr.drop(columns='weather')
hr

,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,precipitation,id,main,description,icon
dt,,,,,,,,,,,,,,,,,
2021-05-05 12:00:00,8.21,3.37,996,81,5.15,0.89,75,10000,12.23,282,16.99,1.00,0.20,500,Rain,light rain,10d
2021-05-05 13:00:00,7.81,2.81,996,83,5.11,1.37,80,10000,12.31,279,17.87,0.80,0.00,803,Clouds,broken clouds,04d
2021-05-05 14:00:00,7.52,2.33,997,84,5.00,1.07,85,10000,12.79,279,18.64,0.80,0.00,804,Clouds,overcast clouds,04d
2021-05-05 15:00:00,7.75,2.57,998,83,5.05,0.72,89,10000,13.15,279,19.08,1.00,0.24,500,Rain,light rain,10d
2021-05-05 16:00:00,7.21,1.91,1000,83,4.52,0.42,94,10000,12.81,279,18.81,1.00,0.13,500,Rain,light rain,10d
2021-05-05 17:00:00,6.47,1.08,1002,82,3.63,0.19,93,10000,11.95,278,17.89,0.79,0.00,804,Clouds,overcast clouds,04d
2021-05-05 18:00:00,5.93,0.59,1002,79,2.59,0.06,88,10000,10.94,276,16.45,0.75,0.00,804,Clouds,overcast clouds,04d
2021-05-05 19:00:00,5.46,0.18,1003,75,1.40,0.00,29,10000,10.08,276,15.41,0.13,0.00,802,Clouds,scattered clouds,03d
2021-05-05 20:00:00,5.41,0.23,1003,72,0.82,0.00,25,10000,9.65,275,14.76,0.15,0.00,802,Clouds,scattered clouds,03n


## Daily Data
- 1 week
- 8 observations

In [7]:
time = ['dt', 'sunrise', 'sunset']
dly = pd.DataFrame(data['daily'])
dly[time] = dly[time].apply(pd.to_datetime, unit='s')

dly = dly.join(pd.DataFrame(dly.temp.to_list()))
dly = dly.join(pd.DataFrame(dly.feels_like.to_list()).add_suffix('_feels_like'))
dly = dly.join(pd.DataFrame(dly.weather.apply(del_lst).to_list()))

dly = dly.drop(columns=['temp', 'feels_like', 'weather'])
dly = dly.set_index('dt')
dly

,sunrise,sunset,moonrise,moonset,moon_phase,pressure,humidity,dew_point,wind_speed,wind_deg,...,eve,morn,day_feels_like,night_feels_like,eve_feels_like,morn_feels_like,id,main,description,icon
dt,,,,,,,,,,,,,,,,,,,,,
2021-05-05 11:00:00,2021-05-05 03:46:14,2021-05-05 19:05:35,1620181620,1620215280,0.80,996,83,5.20,13.15,279,...,6.47,5.29,3.25,-0.52,1.08,-0.52,501,Rain,moderate rain,10d
2021-05-06 11:00:00,2021-05-06 03:44:20,2021-05-06 19:07:22,1620268980,1620306180,0.83,1006,59,1.67,9.00,272,...,7.68,3.48,5.47,-0.90,5.72,-0.90,500,Rain,light rain,10d
2021-05-07 11:00:00,2021-05-07 03:42:27,2021-05-07 19:09:09,1620356160,1620396960,0.87,1010,71,2.63,7.97,289,...,7.78,2.80,3.92,-0.31,4.08,-0.31,500,Rain,light rain,10d
2021-05-08 11:00:00,2021-05-08 03:40:36,2021-05-08 19:10:54,1620443280,1620487680,0.90,1013,56,3.57,8.21,186,...,8.45,5.27,10.60,2.47,4.75,2.47,501,Rain,moderate rain,10d
2021-05-09 11:00:00,2021-05-09 03:38:47,2021-05-09 19:12:40,1620530400,1620578280,0.93,1001,86,15.00,8.50,217,...,18.08,12.73,17.27,12.51,18.06,12.51,500,Rain,light rain,10d
2021-05-10 11:00:00,2021-05-10 03:36:59,2021-05-10 19:14:24,1620617520,1620668940,0.96,1003,96,11.21,5.72,215,...,11.47,13.32,11.52,13.16,11.00,13.16,501,Rain,moderate rain,10d
2021-05-11 11:00:00,2021-05-11 03:35:13,2021-05-11 19:16:08,1620704700,1620759600,0.00,1010,71,9.57,4.90,212,...,10.68,10.51,14.16,9.99,10.05,9.99,500,Rain,light rain,10d
2021-05-12 11:00:00,2021-05-12 03:33:29,2021-05-12 19:17:51,1620792060,1620850320,0.02,1015,64,6.57,4.85,310,...,9.54,9.86,12.24,9.55,7.63,9.55,500,Rain,light rain,10d
